

The new openinsider data is scraped into a csv

The cell 'Initial cleaning' from creating training data set ipynb is run on that data to load in the csv and clean.

Ticker prices are downloaded for each symbol, prev 100 days and the following 35 or something 

Ticker prices are put into the new trades using the 'Putting 90days of prices into df_by_ticker' cell

prev day of SPY history is downloaded.

The create_features() def is run to create all the new features.